# LangChain Expression Language (LCEL)

A Linguagem de Expressão LangChain, ou LCEL, é uma forma declarativa de **compor cadeias de maneira fácil**. A LCEL foi projetada desde o primeiro dia para suportar a colocação de protótipos em produção, sem a necessidade de alterações no código, desde a cadeia mais simples “prompt + LLM” até as cadeias mais complexas (já vimos pessoas executando com sucesso cadeias LCEL com centenas de etapas em produção). Para destacar alguns dos motivos pelos quais você pode querer usar a LCEL:
- **Suporte a streaming de primeira classe**: menor tempo possível para saída do primeiro token produzidio;
- **Suporte assíncrono**: Qualquer cadeia construída com a LCEL pode ser chamada tanto com a API síncrona;
- **Execução paralela otimizada**: Sempre que suas cadeias LCEL tiverem etapas que podem ser executadas em paralelo, automaticamente é feito isso;
- **Retentativas e fallbacks**: É maneira de tornar suas cadeias mais confiáveis em grande escala, na qual ações alternativas podem ser tomadas no caso de um erro em uma cadeia
- **Acessar resultados intermediários**: auxiliando na depuração de uma cadeia;

## Um exemplo simples de LCEL

O exemplo mais simples que podemos mostrar seria na construção de *prompt + model*

In [ ]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o seguinte assunto: {assunto}')

In [ ]:
chain = prompt | model
chain.invoke({'assunto': 'futebol'})

### Adicinando mais elementos a chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
chain.invoke({'assunto': 'futebol'})

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o seguinte: {assunto}')

chain = prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'assunto': 'gatinhos'})

### ATENÇÃO! A ordem importa

In [ ]:
chain =  ChatOpenAI() | prompt | StrOutputParser()
chain.invoke({'assunto': 'gatinhos'})

### Como eu sei a ordem?

Para atingir o mesmo objetivo sem a criação da chain, os passos que eu deveria seguir seriam:
- Formatar o prompt template
- Enviar o prompt formatado para o modelo
- Fazer o parseamento do saída do modelo
Essa mesma ordem deve ser seguida para não termos erros

In [ ]:
input = {'assunto': 'gatinhos'}

In [ ]:
prompt_formatado = prompt.invoke(input)
prompt_formatado

In [ ]:
resposta = model.invoke(prompt_formatado)
resposta

In [ ]:
StrOutputParser().invoke(resposta)

### Para não errar a ordem

É importante entendermos que cada componente recebe um tipo de entrada e gera um tipo de saída, e estes tipos precisam casar:

| Component       | Tipo de Entrada                                    | Tipo de Saída        |
|-----------------|-----------------------------------------------------|----------------------|
| Prompt          | Dicionário                                         | PromptValue      |
| ChatModel       | String única, lista de mensagens de chat ou PromptValue | Mensagem de Chat     |
| LLM             | String única, lista de mensagens de chat ou PromptValue | String               |
| OutputParser    | A saída de um LLM ou ChatModel                     | Depende do parser    |
| Retriever       | String única                                       | Lista de Documentos  |
| Tool            | String única ou dicionário, dependendo da ferramenta| Depende da ferramenta|

Podemos verificar isso pelos output e input schemas:

In [ ]:
prompt.input_schema.model_json_schema()

In [ ]:
prompt.output_schema.model_json_schema()

### Como eu teria feito com chains clássicas?

In [ ]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")
output_parser = StrOutputParser()

In [ ]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser
)
chain.invoke({'assunto': 'gatinhos'})